In [89]:
#Load Train dataset
import pandas as pd
train_data = pd.read_csv("https://raw.githubusercontent.com/AtharvaKalsekar/Kaggle/master/titanic/train.csv")
print(train_data.columns)

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [0]:
print(train_data.dtypes)

In [90]:
train_data.drop(columns=["PassengerId","Name","Ticket","Cabin","Embarked"],inplace=True)
print(train_data.dtypes)

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Fare        float64
dtype: object


In [91]:
#Check for missing values
train_data.isnull().any()

Survived    False
Pclass      False
Sex         False
Age          True
SibSp       False
Parch       False
Fare        False
dtype: bool

In [0]:
# Get the numpy arrays for independent and target variables
import numpy as np
X = train_data.iloc[:,1:].values
y = train_data.iloc[:,0].values

In [98]:
print(X.shape)
print(y.shape)

(891, 6)
(891,)


In [0]:
# Label encoding the strings
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,1] = le.fit_transform(X[:,1])

In [0]:
#treat missing values for columns obtained from dataframe.isnull().any()
from fancyimpute import KNN
impt=KNN(k=3)
X=impt.fit_transform(X)

In [0]:
#scale the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X = sc.fit_transform(X)

In [0]:
#Spliting train and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X,y,test_size = 0.3, random_state=0, stratify=y)

In [102]:
print(X_train.shape)
print(y_train.shape)

(623, 547)
(623,)


In [0]:
#PCA
from sklearn.decomposition import KernelPCA
pca = KernelPCA(n_components=None, kernel="rbf")
X_train = pca.fit_transform(X_train)
X_val = pca.transform(X_val)

In [0]:
#load model KNN
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors = 5, p = 2, metric = "minkowski")

In [104]:
#fit model
model.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

In [105]:
#Evaluate model
from sklearn.model_selection import cross_val_score
acc = cross_val_score(estimator = model, X = X_train, y = y_train, cv=10)
print(acc.mean())

0.8042695978444259


In [111]:
#Processing test data
test_data = pd.read_csv("https://raw.githubusercontent.com/AtharvaKalsekar/Kaggle/master/titanic/test.csv")
print(test_data.dtypes)

PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object


In [0]:
testset_ids = test_data[["PassengerId"]]
testset = test_data.drop(columns = ["PassengerId","Name","Ticket","Cabin","Embarked"])

In [85]:
testset.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare'], dtype='object')

In [0]:
testset.describe

In [82]:
#check for missing values
testset.isnull().any()

Pclass    False
Sex       False
Age        True
SibSp     False
Parch     False
Fare       True
dtype: bool

In [113]:
#Get the numpy arrays for test set
testset = testset.iloc[:,:].values
print(testset.shape)

(418, 6)


In [0]:
#label encode test set
testset[:,1] = le.transform(testset[:,1])

In [0]:
#treat missing values
testset = impt.fit_transform(testset)

In [0]:
#feature scaling for test set
testset = sc.transform(testset)

In [0]:
#PCA for test set
testset = pca.transform(testset)

In [0]:
#predict
testset_pred = model.predict(testset)

In [0]:
#storing the result
result_df=pd.DataFrame(testset_pred)
result_df=pd.concat([testset_ids,result_df], axis=1,ignore_index=True)
print(result_df)

In [0]:
result_df.to_csv('out.csv')